# 基础rag

In [ ]:
# 构建提示词
def question_answering(context, query):
    prompt = "使用{context}中的信息来回答{query}中的用户问题。如果所提供的\
    上下文中没有相关信息，就尝试自己回答，但是要告诉用户你的回答没有任何相关的上下文依据。请简洁明了，回答的内容不得超过80个词。"
    response = get_completion(instruction, prompt, model="gpt-3.5-turbo")
    answer = response.choices[0].message["content"]
    return answer


# 先进的rag

In [ ]:
# 构建提示词
question = "对于分类问题最好的机器学习算法是什么?"
retrieved_docs = ["文档1: SVM算法被广泛应用于 ...", "文档2:随机森林算法是一种鲁棒性很高的..."]
template = "请基于这些文档{docs}，帮助回答以下问题{question}"
full_prompt = template.format(question=question, docs=" ".join(retrieved_docs))

# 大模型主导的rag

In [ ]:
# 头文件
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain import LLMChain
from langchain.utilities import BingSearchAPIWrapper
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import os

# 工具
os.environ["BING_SEARCH_URL"] = "xxx"
os.environ["BING_SUBSCRIPTION_KEY"] = "xxx"
search = BingSearchAPIWrapper()
tools = [ Tool( name = "Search", func=search.run,
        description="useful for when you need to answer questions about current events"
            )
]

#提示词模板
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

# 大语言模型
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "xxx"
os.environ["OPENAI_API_KEY"] = "xxx"
llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", openai_api_version="2023-03-15-preview")

# 输出解析器
class CustomOutputParser(AgentOutputParser):
        def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
            if "Final Answer:" in llm_output:
                return AgentFinish(
                    return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                    log=llm_output,
                )
            regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
            match = re.search(regex, llm_output, re.DOTALL)
            if not match:
                raise ValueError(f"Could not parse LLM output: `{llm_output}`")
            action = match.group(1).strip()
            action_input = match.group(2)
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
output_parser = CustomOutputParser()

# Agent
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

# 执行
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("generate a short blog post to review the plot of the movie Avatar 2. It should be a positive review.")